In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
mnist = input_data.read_data_sets('/tmp/data/', one_hot= True)

In [0]:
num_steps = 100000
bsize = 128
learning_rate = 0.001

image_d = 784 
ghd = 256
dhd = 256
nd = 100

In [0]:
G = tf.placeholder(tf.float32,[None,nd])

In [0]:
D = tf.placeholder(tf.float32,[None,image_d])

In [0]:
def rand(shape):
  return tf.random_normal(shape,stddev=1.0)

In [0]:
weights={'g1': tf.Variable(rand([nd, ghd])),'g2': tf.Variable(rand([ghd, image_d])),
         'd1':tf.Variable(rand([image_d, dhd])),'d2':tf.Variable(rand([dhd, 1]))}

In [0]:
biases={'g1': tf.Variable(tf.zeros([ghd])),'g2': tf.Variable(tf.zeros([image_d])),
         'd1':tf.Variable(tf.zeros([dhd])),'d2':tf.Variable(tf.zeros([1]))}

In [0]:
def gen(x):
  hl = tf.nn.relu(tf.add(tf.matmul(x,weights['g1']),biases['g1']))
  ol = tf.nn.sigmoid(tf.add(tf.matmul(hl,weights['g2']),biases['g2']))
  return ol
  

In [0]:
def disc(x):
  hl = tf.nn.relu(tf.add(tf.matmul(x,weights['d1']),biases['d1']))
  ol = tf.nn.sigmoid(tf.add(tf.matmul(hl,weights['d2']),biases['d2']))
  return ol

In [0]:
Greal = gen(G)

In [0]:
Dreal = disc(D)
Dfake = disc(Greal)

In [0]:
Gloss = tf.reduce_mean(tf.log(Dfake))

In [0]:
Dloss = tf.reduce_mean(tf.log(Dreal)+tf.log(1.0-Dfake))

In [0]:
optimizerG = tf.train.AdamOptimizer(learning_rate)

In [0]:
optimizerD = tf.train.AdamOptimizer(learning_rate)

In [0]:
Dvars = [weights['d1'], weights['d2'], biases['d1'], biases['d2']]

In [0]:
Gvars = [weights['g1'], weights['g2'],biases['g1'], biases['g2']]

In [0]:
trainG = optimizerG.minimize(Gloss,var_list=Gvars)

In [0]:
trainD = optimizerG.minimize(Dloss,var_list=Dvars)

In [0]:
init = tf.global_variables_initializer()

In [0]:
with tf.Session() as sess:
  sess.run(init)
  for i in range(1,num_steps+1):
    Xbatch,Ybatch = mnist.train.next_batch(bsize)
    noise = np.random.uniform(-1.0,1.0,size=[bsize,nd])
    feed_dict = {D: Xbatch, G: noise}
    _, _, gl, dl = sess.run([trainG, trainD, Gloss, Dloss],feed_dict=feed_dict)
    if i % 1000 == 0 or i == 1:
      print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))